In [1]:

from functools import partial
import pandas as pd
import time
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import RandomForestRegressor as rfr
import shap
import xgboost as xgb
import numpy as np

### Models

In [2]:
from BioML.features import selection
from pycaret.datasets import get_data
import pandas as pd
from sklearn.model_selection import train_test_split
from pathlib import Path

In [3]:
data = get_data('diabetes')
new_data = data.drop("Class variable", axis=1)
labels = data["Class variable"]

,Number of times pregnant,Plasma glucose concentration a 2 hours in an oral glucose tolerance test,Diastolic blood pressure (mm Hg),Triceps skin fold thickness (mm),2-Hour serum insulin (mu U/ml),Body mass index (weight in kg/(height in m)^2),Diabetes pedigree function,Age (years),Class variable
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
ester_label = pd.read_csv("../data/esterase_labels.csv", index_col=0)
ester = "-../data/esterase_features.xlsx"

## Classification

In [5]:
import BioML.models.classification as classification
from collections import defaultdict
from functools import partial
import pandas as pd
import numpy as np

In [6]:
plot = ("learning", "confusion_matrix", "class_report")

In [7]:
excel = pd.read_excel("../data/esterase_features.xlsx", index_col=0, sheet_name="ch2_20")
label = pd.read_csv("../data/esterase_labels.csv", index_col=0)
l = pd.Series(label["label1"])

In [8]:
data = classification.DataParser("../data/esterase_features.xlsx", "../data/esterase_labels.csv", sheets="ch2_20")
experiment = classification.PycaretInterface("classification", 200, budget_time=20, best_model=3, 
                                  output_path="classification_training", optimize="MCC")
regressor = classification.Classifier(test_size=0.2, optimize="MCC")
training = classification.Trainer(experiment, regressor, 5, 30)


27-03-2024 14:06:26 INFO ------------------------------------------------------------------------------
27-03-2024 14:06:26 INFO PycaretInterface parameters
27-03-2024 14:06:26 INFO Seed: 200
27-03-2024 14:06:26 INFO Budget time: 20
27-03-2024 14:06:26 INFO The number of models to select: 3
27-03-2024 14:06:26 INFO Output path: classification_training
27-03-2024 14:06:26 INFO ----------------Trainer inputs-------------------------
27-03-2024 14:06:26 INFO Number of kfolds: 5
27-03-2024 14:06:26 INFO Number of iterations: 30


In [11]:
c = classification.split.ClusterSpliter("../data/resultsDB_clu.tsv", 2, random_state=experiment.seed)
X_train, X_test = c.train_test_split(data.features)

In [12]:
results, models = training.generate_training_results(X_train, data.label, tune=True, 
                                                           test_data=X_test, fold_strategy=c)

27-03-2024 14:14:49 INFO --------------------------------------------------------
27-03-2024 14:14:49 INFO Training classification models
27-03-2024 14:14:49 INFO The models used ['rf', 'lda', 'rbfsvm', 'et', 'dt', 'knn', 'xgboost', 'nb', 'mlp', 'catboost', 'ridge', 'dummy', 'svm', 'lr', 'qda', 'gbc']
27-03-2024 14:14:49 INFO Time budget is 20 minutes
27-03-2024 14:14:55 INFO Model mlp trained in 0.1 minutes


Model mlp trained in 0.1 minutes


27-03-2024 14:14:59 INFO Model dummy trained in 0.065 minutes


Model dummy trained in 0.065 minutes


27-03-2024 14:15:03 INFO Model knn trained in 0.07 minutes


...

27-03-2024 14:15:43 INFO Model catboost trained in 0.123 minutes


Model catboost trained in 0.123 minutes


27-03-2024 14:15:45 INFO Model ridge trained in 0.04 minutes
27-03-2024 14:15:45 INFO Training over: Total runtime 0.934 minutes
27-03-2024 14:15:45 INFO Analyse the best models and plotting them
27-03-2024 14:15:45 INFO Analyse the top 1 model: knn


Model ridge trained in 0.04 minutes


27-03-2024 14:15:47 INFO Analyse the top 2 model: et
27-03-2024 14:15:49 INFO Analyse the top 3 model: rbfsvm
27-03-2024 14:15:50 INFO --------Stacking the best models--------
27-03-2024 14:15:50 INFO ----------Stacking the best models--------------
27-03-2024 14:15:54 INFO --------Creating an ensemble model--------
27-03-2024 14:15:54 INFO ----------Creating a majority voting model--------------
27-03-2024 14:15:54 INFO fold: 5
27-03-2024 14:15:54 INFO weights: None
27-03-2024 14:15:57 INFO --------Retuning the best models--------
27-03-2024 14:15:57 INFO Retuning knn
27-03-2024 14:15:57 INFO ---------Retuning the best models--------------
27-03-2024 14:15:57 INFO num_iter: 30
27-03-2024 14:15:57 INFO fold: 5
27-03-2024 14:16:04 INFO Retuning et
27-03-2024 14:16:04 INFO ---------Retuning the best models--------------
27-03-2024 14:16:04 INFO num_iter: 30
27-03-2024 14:16:04 INFO fold: 5
27-03-2024 14:16:25 INFO Retuning rbfsvm
27-03-2024 14:16:25 INFO ---------Retuning the best models

##### Acces the ClaasificationExperiment from Pycaret and play with teh models if you want

In [8]:
experiment.pycaret

In [11]:
test_set_predictions = training.generate_holdout_prediction(models)

In [14]:
training_output = "classification_results"
l = []
for tune_status, result_dict in results.items():
    for key, value in result_dict.items():
        classification.write_results(f"{training_output}/{tune_status}", *value, sheet_name=key)
    classification.write_results(f"{training_output}/{tune_status}", test_set_predictions[tune_status] , sheet_name=f"test_results")

## Generate Models

In [16]:
from BioML.models import save_model

In [17]:
data = save_model.DataParser("../data/esterase_features.xlsx", "../data/esterase_labels.csv", sheets="ch2_20")
experiment = save_model.PycaretInterface("classification", 200, budget_time=20, best_model=3, 
                                  output_path="classification_training", optimize="MCC", experiment_name="generate_model")
regressor = save_model.Classifier(test_size=0.2, optimize="MCC", selected="lr")
training = save_model.Trainer(experiment, regressor, 5, 30)


27-03-2024 14:28:52 INFO ------------------------------------------------------------------------------
27-03-2024 14:28:52 INFO PycaretInterface parameters
27-03-2024 14:28:52 INFO Seed: 200
27-03-2024 14:28:52 INFO Budget time: 20
27-03-2024 14:28:52 INFO The number of models to select: 3
27-03-2024 14:28:52 INFO Output path: classification_training
27-03-2024 14:28:52 INFO ----------------Trainer inputs-------------------------
27-03-2024 14:28:52 INFO Number of kfolds: 5
27-03-2024 14:28:52 INFO Number of iterations: 30


In [18]:
regressor.selected

'lr'

In [19]:
sorted_results, sorted_models, top_params = training.run_training(data.features, data.label)

2024/03/27 14:28:55 INFO mlflow.tracking.fluent: Experiment with name 'generate_model' does not exist. Creating a new experiment.
27-03-2024 14:28:56 INFO --------------------------------------------------------
27-03-2024 14:28:56 INFO Training classification models
27-03-2024 14:28:56 INFO The models used ['lr']
27-03-2024 14:28:56 INFO Time budget is 20 minutes
27-03-2024 14:29:01 INFO Model lr trained in 0.092 minutes
27-03-2024 14:29:01 INFO Training over: Total runtime 0.092 minutes


Model lr trained in 0.092 minutes


In [20]:
generate = save_model.GenerateModel(training)
models =  generate.train_by_strategy(sorted_models, "simple:0")
final_model = generate.finalize_model(models)
generate.save_model(final_model, "model_output/logistic")

27-03-2024 14:29:02 INFO ----------Finalizing the model by training it with all the data including test set--------------


Transformation Pipeline and Model Successfully Saved


## Prediction

In [13]:
from BioML.models import predict
import pandas as pd
import numpy as np
from scipy.spatial import distance

In [21]:
training_features = "../data/esterase_features.xlsx"
label = "../data/esterase_labels.csv"
test_features = "../data/esterase_features.xlsx"
outlier_train=()
outlier_test=()
sheet_name="ch2_20"
problem="classification"
model_path="model_output/logistic"
scaler="zscore"

In [23]:
test_features

,hydrophobicity_FASG890101.2.residue25,alphaticr.negativecharger.gap5,BHAR880101.lag9_moran,tri_gram_pssm7077,tri_gram_pssm286,tri_gram_pssm466,tri_gram_pssm3493,tri_gram_pssm1298,tri_gram_pssm5728,tri_gram_pssm7468,tri_gram_pssm1297,tri_gram_pssm4023,tri_gram_pssm3159,tri_gram_pssm7678,tri_gram_pssm7234,tri_gram_pssm5801,tri_gram_pssm5367,tri_gram_pssm7177
#,,,,,,,,,,,,,,,,,,
EH1(72),22.857143,0.055016,-0.102575,0.065363,0.832237,0.442684,0.180234,0.089845,0.490688,0.179098,0.027139,0.624236,0.273944,0.460827,0.272559,0.612637,0.401477,0.029163
EH2(71),25.513196,0.053731,-0.068227,0.067360,0.645581,0.449828,0.113174,0.094496,0.332225,0.160173,0.024556,0.627025,0.241681,0.361142,0.294981,0.617432,0.372408,0.028312
EH3(69),27.873563,0.055556,-0.010242,0.021795,0.756387,0.481429,0.207077,0.123859,0.046267,0.153389,0.032419,0.578253,0.229875,0.456541,0.315422,0.460896,0.360463,0.011199
CalB(68),21.806854,0.034921,-0.083474,0.006633,0.097950,0.037538,0.015682,0.006383,0.026272,0.007791,0.003719,0.093919,0.022174,0.039728,0.034692,0.132978,0.032155,0.005397
EH4(67),25.730994,0.053571,-0.103226,0.065890,0.685630,0.458253,0.117385,0.102536,0.358012,0.155480,0.025488,0.620764,0.234235,0.351025,0.277734,0.608569,0.365297,0.027124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
EH141(1),26.797386,0.050773,-0.079524,0.006679,0.134293,0.070896,0.014058,0.003704,0.031492,0.015362,0.001344,0.183355,0.036559,0.034975,0.038975,0.141937,0.026144,0.003584
EH142(1),32.676056,0.034384,0.002960,0.001735,0.103232,0.054376,0.016582,0.007587,0.015728,0.009787,0.008807,0.106868,0.027981,0.123676,0.035153,0.083529,0.015107,0.001517
EH143(1),27.921093,0.064319,0.048037,0.005556,0.089063,0.102606,0.022504,0.006974,0.024215,0.031432,0.002388,0.184709,0.029490,0.134652,0.072092,0.177750,0.035943,0.004516


In [26]:
feature = predict.DataParser(training_features, label, outliers=outlier_train, sheets=sheet_name)
test_features = feature.remove_outliers(feature.read_features(test_features, "ch2_20"), outlier_test)
predictions = predict.predict(test_features, model_path, problem)


In [44]:
transformed, scaler_dict, test_x = predict.scale(scaler, feature.drop(), test_features)
domain = predict.ApplicabilityDomain()
domain0 = predict.domain_filter(predictions, transformed, test_x, 5)

In [45]:
fasta = "../data/whole_sequence.fasta"
res_dir = "prediction_results_domain"

In [46]:
predictions.index = [f"sample_{x}" for x, _ in enumerate(predictions.index)]

In [47]:
col_name = ["prediction_score", "prediction_label", "AD_number"]
predictions = predictions.loc[:, predictions.columns.str.contains("|".join(col_name))]

In [48]:
extractor = predict.FastaExtractor(fasta, res_dir)
positive, negative = extractor.separate_negative_positive(domain0)


In [49]:
extractor.extract(positive, negative, positive_fasta=f"positive.fasta", negative_fasta=f"negative.fasta")